# Optimal Activity Levels

We consider the selection of $n$ nonnegative activity levels, denoted $x_1 , \ldots , x_n$. These activities consume $m$ resources, which are limited. Activity $j$ consumes $A_{ij} x_j$ of resource $i$, where $A_{ij}$ are given. The total resource consumption is additive, so the total of resource $i$ consumed is $c_i = \sum_{j=1}^n A_{ij} x_j$ . (Ordinarily we have $A_{ij} \geq 0$, i.e., activity $j$ consumes resource $i$. But we allow the possibility that $A_{ij} < 0$, which means that activity $j$ actually generates resource $i$ as a by-product.) Each resource consumption is limited: we must have $c_i \leq c_i^{max}$, where $c_i^{max}$ are given. Each activity generates revenue, which is a piecewise-linear concave function of the activity level:

\begin{align}
r_j(x_j) = 
     \begin{cases}
       p_j x_j & \quad 0 \leq x_j \leq q_j\\
       p_j x_j + p_j^{disc}(x_j - q_j) &\quad x_j \geq q_j\\
     \end{cases}
\end{align}

Here $p_j > 0$ is the basic price, $q_j > 0$ is the quantity discount level, and $p_j^{disc}$ is the quantity discount price, for (the product of) activity $j$. (We have $0 < p_j^{disc} < p_j$ .) The total revenue is the sum of the revenues associated with each activity, i.e., $\sum_{j=1}^n r_j(x_j)$. 

**The goal is to choose activity levels that maximize the total revenue while respecting the resource limits.**

## Problem data

Consider the following problem data,

- $A \in \mathbb{R}^{m \times n}$, where $A_{ij}$ is the amount of resource $i$ consumed/produced by unity of activity $j$
- $c^{max} \in \mathbb{R}^m_+$, where $c_i$ is the maximum resource comsuption alowed for resource $i$
- $p \in \mathbb{R}^n_+$, where $p_j$ is the basic price for activity $j$
- $p^{disc} \in \mathbb{R}^n_+$, where $p_j^{disc}$ is the discount price for activity $j$
- $q \in \mathbb{R}^n_+$, where $q_j$ is the quantity discount level for activity $j$


In [63]:
import numpy as np
np.set_printoptions(precision=2, suppress=True)
from cvxpy import *

A = np.array([[1, 2, 0, 1], [0, 0, 3, 1],[0, 3, 1, 1], [2, 1, 2, 5], [1, 0, 3, 2]], dtype=np.float)
cmax = np.array([100, 100, 100, 100, 100],dtype=np.float)
p = np.array([3, 2, 7, 6],dtype=np.float)
pdisc = np.array([2, 1, 4, 2],dtype=np.float)
q = np.array([4, 10, 5, 10],dtype=np.float)



## Problem Description

The problem can be directly expressed as, 

\begin{equation*}
\begin{aligned}
& \underset{x}{\text{maximize}}
& & \sum_{j=1}^n r_j(x_j) \\
& \text{subject to}
& & x \succeq 0 \\
& & & Ax \preceq c^{max} \\
\end{aligned}
\end{equation*}

This is a convex optimization problem since the objective is concave and constraints are a set of linear inequalities. Note the function $r_j(\cdot)$, can be described as pointwise minimum of the two inner functions since maximizing the minimum of them is the same as maximizing $r_j(\cdot)$.  In fact, this problem can be expressed as Linear Program (LP), but let the CVXPY do this job for us.


In [79]:
# Revenue Function
def revenue(x):
    x_ravel = np.ravel(x)
    ind1 = (x_ravel < q).astype(np.int)
    ind2 = (x_ravel >= q).astype(np.int)
    return (ind1 * p * x_ravel) + (ind2 * (p * q + pdisc * (x_ravel-q)))


# Define the problem
x = Variable(A.shape[1])
objective = Maximize(sum(min_elemwise(mul_elemwise(p, x), mul_elemwise(p, q) + mul_elemwise(pdisc, (x-q)))))
constraints = [x >= 0, A*x <= cmax]
prob = Problem(objective, constraints)


# Solve and print results
result = prob.solve()
print("Optimal revenue p* = {:.2f}".format(result))
print("Optimal activity levels x* = {}".format(np.ravel(x.value)))
print("Optimal revenue per activity r_j: {}".format(revenue(x.value)))
print("Average price per unit of activity: {}".format(revenue(x.value)/np.ravel(x.value)))
print("Total Revenue: {:.2f}".format(revenue(x.value).sum()))

Optimal revenue p* = 192.50
Optimal activity levels x* = [  4.   22.5  31.    1.5]
Optimal revenue per activity r_j: [  12.    32.5  139.     9. ]
Average price per unit of activity: [ 3.    1.44  4.48  6.  ]
Total Revenue: 192.50
